In [9]:
import io
import os
import sys
import math

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import collections

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

MSLR-WEB10K
Ranklib



In [10]:
def load_txt(foldDir,foldsize,foldnum):
    corpusDir = os.path.join('%s%s' % (inputDir,  os.listdir(foldDir)[foldsize]))
    dataDir = os.path.join('%s/%s' % (corpusDir, os.listdir(corpusDir)[foldnum]))
    trainDir = os.path.join('%s/%s' % (dataDir, "train.txt"))
    testDir = os.path.join('%s/%s' % (dataDir, "test.txt"))
    validDir = os.path.join('%s/%s' % (dataDir, "vali.txt"))
    return [trainDir,testDir ,validDir]

# Dropping column with 100% nan values (parsing problem probably)
def drop_col(df):
    df.drop(df.columns[-1], axis=1, inplace=True)

# Cleaning values
def split_semicolon(df):
    # removes string naming pattern '*:' from values
    for col in range(1,len(df.columns)):
        df.loc[:,col] = df.loc[:,col].apply(lambda x: str(x).split(':')[1])
    df.columns = ['rel', 'qid'] + [str(x) for x in range(1,137)] # renaming cols

In [11]:
print("done")

done


In [12]:
def load_csv(foldDir,foldsize,foldnum):
    corpusDir = os.path.join('%s%s' % (inputDir,  os.listdir(foldDir)[foldsize]))
    dataDir = os.path.join('%s/%s' % (corpusDir, os.listdir(corpusDir)[foldsize]))
    trainDir = os.path.join('%s/%s' % (dataDir, "train.csv"))
    testDir = os.path.join('%s/%s' % (dataDir, "test.csv"))
    validDir = os.path.join('%s/%s' % (dataDir, "vali.csv"))
    return [trainDir,testDir ,validDir]

inputDir = "../input/"
dataDir = load_csv(inputDir,1,1)

trainset = pd.read_csv(dataDir[0])
trainset.drop(trainset.columns[0], axis=1, inplace=True)
testset  = pd.read_csv(dataDir[1])
testset.drop(testset.columns[0], axis=1, inplace=True)
validset = pd.read_csv(dataDir[2])
validset.drop(validset.columns[0], axis=1, inplace=True)

# trainset


In [13]:
class Normarlization():
    def __init__(self,train_input,valid_input,test_input,concate_normalize=False):
        if concate_normalize == True:
            whole_data = np.concatenate((train_input,valid_input,train_input),axis=0)
        else:
            whole_data = train_input
        self.normalize_factor = np.max(whole_data,axis=0)
        self.normalize_factor[self.normalize_factor==0] = 1
            
    def perform_normalize(self,input_data):
        return np.divide(input_data,self.normalize_factor)

In [14]:
import random
from numpy import inf

def NDCG_metric(dataset,predict,target,position):
    Result_predict = collections.defaultdict(int)
    Result_target = collections.defaultdict(int)
    Result_normalization = collections.defaultdict(int)

    index = 0
    
    qid = qid_old = dataset['qid'][0]
    for i in range(len(dataset)): 
        qid = dataset['qid'][i]
        
        if qid != qid_old:
            index = 0
        
        if index < position:
            Result_predict[qid] += (np.power(2, predict[i])-1)/(np.log2(index+2))
            Result_target[qid] += (np.power(2,  target[i])-1)/(np.log2(index+2))
            index += 1
            qid_old = qid
       
    for key in Result_predict.keys():
        Result_normalization[key] = Result_predict[key]/Result_target[key] if Result_target[key] > 0 else 0
     
    data = [Result_predict.values(),Result_target.values(),Result_normalization.values()]
    df = pd.DataFrame(data,index=['predict','target','NDCG'],columns=Result_predict.keys())
    return df.T


def sigmoid_f(z):
    z = -z
    z = np.exp(z)
    z = 1+z
    z = 1/z
    return z

def softmax(x):
#     print(np.exp(x))
    # Robust Softmax
    exp_x = np.exp(x)
    exp_x[exp_x==inf] = 1e10
    return (exp_x.T / np.sum(exp_x, axis=1)).T

def logistic_f(X_train, y_train, X_validation=None, y_validation=None, X_test=None, y_test=None, accuracy_period=1, model_path="../model/logistic_w.npy", learning_rate=0.00001, batch_size=64, number_of_runs=1000, lambda_v=0):
    nomarlizer = Normarlization(train_input=X_train, valid_input=X_valid, test_input=X_test,concate_normalize=False)
    print("normalizing train")
    X_train = nomarlizer.perform_normalize(X_train)
    X_train = np.append(X_train, np.ones((X_train.shape[0], 1)), axis=1)
    n_data = X_train.shape[0]
    print(X_train.shape,y_train.shape)
    w = np.random.rand(X_train.shape[1], y_train.shape[1]) * 2 - 1

    try:
        np.save(model_path, w)
    except Exception as e:
        print("The save path doesn't exist, creating a new one...")
#         print(e)
#         import sys
#         sys.exit()
        os.makedirs(model_path)



    m = batch_size
    test_accuracy = ""
    validation_accuracy = ""
    
    test_flag = 0
    if type(X_test) != type(None) and type(y_test) != type(None):
        print("normalizing test")
        test_flag = 1
        X_test = nomarlizer.perform_normalize(X_test)
        X_test = np.append(X_test, np.ones((X_test.shape[0], 1)), axis=1)


    validation_flag = 0
    if type(X_validation) != type(None) and type(y_validation) != type(None):
        print("normalizing validation")
        validation_flag = 1
        X_validation = nomarlizer.perform_normalize(X_validation)
        X_validation = np.append(X_validation, np.ones((X_validation.shape[0], 1)), axis=1)


    print("start training")
    prev_ndcg = 0
    test_result_accuracy = 0
    for i in range(number_of_runs):
        random_list = random.sample(range(n_data), batch_size)
#         x_batch = X_train[random_list]
#         y_batch = y_train[random_list]
        
        x_batch = X_train
        y_batch = y_train
        
        #logistic core codes
        y_pre = softmax(np.matmul(x_batch, w))
        delta = (y_pre - y_batch).T
        D = np.matmul(delta, x_batch).T
        current_learning_rate = learning_rate/math.sqrt(i+1)
        w_grad = learning_rate * (1 / m) * D + (lambda_v / m) * w
        w -= w_grad

        if i % accuracy_period == 0:
            y_pre = softmax(np.matmul(x_batch, w))
            y_pre_value = np.argmax(y_pre, axis=1)
            y_train_value = np.argmax(y_batch, axis=1)
            train_accuracy = sum(np.equal(y_pre_value, y_train_value)) / x_batch.shape[0]

            if test_flag:
                y_test_pre_value = np.argmax(softmax(np.matmul(X_test, w)), axis=1)
                y_test_value = np.argmax(y_test, axis=1)
                test_accuracy = sum(np.equal(y_test_pre_value, y_test_value)) / X_test.shape[0]
                NDCG_test = NDCG_metric(testset,y_test_pre_value,y_test_value,5)
                score = np.mean(NDCG_test['NDCG'])
                print ('The NDCG socre on test set : '  + str(score))
                print(y_test_pre_value[0:30])
                print(y_test_value[0:30])


            if validation_flag:
                y_validation_pre_value = np.argmax(softmax(np.matmul(X_validation, w)), axis=1)
                y_validation_value = np.argmax(y_validation, axis=1)
                validation_accuracy = sum(np.equal(y_validation_pre_value, y_validation_value)) / X_validation.shape[0]
#             print(np.min(np.min(np.log(y_pre),axis=1)))
#             print(np.min(np.min(np.log(np.subtract(1,y_pre)),axis=1)))
            y_pre_log = np.log(y_pre)
            y_pre_log[y_pre_log==-inf] = -5000
            y_pre_log_Complement = np.log(1 - y_pre)
            y_pre_log_Complement[y_pre_log_Complement==-inf] = -5000
#             print(min(min(y_pre_log_Complement)))
            cost = -1 / m * sum(sum((y_batch * y_pre_log + (1 - y_batch) * y_pre_log_Complement)))
            print(str(i).ljust(10) + str(train_accuracy).ljust(30) + str(validation_accuracy).ljust(30)+str(test_accuracy).ljust(30) + str(cost))
            
            if score>0.2 and test_accuracy>0.4:
                if score>prev_ndcg:
                    np.save(model_path, w)
                    prev_ndcg = score
                    test_result_accuracy = test_accuracy
            
#     np.save(model_path, w)
    print("model saved to " + model_path)
    print('The highest Normalized Discounted Culmulative Gain is:',prev_ndcg,'and the corresponding accuracy is:',test_result_accuracy)

In [15]:
trainset_sort = trainset.sort_values(by=['qid','rel'],ascending=False)
validset_sort = validset.sort_values(by=['qid','rel'],ascending=False)
testset_sort = testset.sort_values(by=['qid','rel'],ascending=False)

y_train = trainset_sort['rel'].values
X_train = trainset_sort.drop(['rel','qid'],axis=1).values

y_valid = validset_sort['rel'].values
X_valid = validset_sort.drop(['rel','qid'],axis=1).values

y_test = testset_sort['rel'].values
X_test = testset_sort.drop(['rel','qid'],axis=1).values

y_train_one_hot = np.zeros((y_train.shape[0],5))
y_train_one_hot[np.arange(y_train.shape[0]), y_train] = 1

y_vali_one_hot = np.zeros((y_valid.shape[0],5))
y_vali_one_hot[np.arange(y_valid.shape[0]), y_valid] = 1

y_test_one_hot = np.zeros((y_test.shape[0],5))
y_test_one_hot[np.arange(y_test.shape[0]), y_test] = 1

In [8]:
logistic_f(batch_size=1024, number_of_runs=100, learning_rate=1e-3, accuracy_period=1,
               model_path="./logistic_w_1e-3.npy", X_train=X_train, y_train=y_train_one_hot, X_validation=X_valid,
               y_validation=y_vali_one_hot, X_test=X_test, y_test=y_test_one_hot, lambda_v=0)

normalizing train
(723412, 137) (723412, 5)
normalizing test
normalizing validation
start training


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in exp


The NDCG socre on test set : 0.0536285271556
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:128: RuntimeWarning: divide by zero encountered in log
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:130: RuntimeWarning: divide by zero encountered in log


0         0.512917949937                0.508439634615                0.505380484513                1401664.29237
The NDCG socre on test set : 0.126665775844
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
1         0.431925375858                0.435311720274                0.425035504159                1246180.82148
The NDCG socre on test set : 0.441954067751
[1 0 1 0 0 1 0 1 1 0 2 1 2 0 1 2 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
2         0.395170386999                0.390718314708                0.39160983931                 2117034.45864
The NDCG socre on test set : 0.701766177458
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
3         0.241404621433                0.237793240641                0.240248259986                3031352.99604
The NDCG socre on test set : 0.592396773

The NDCG socre on test set : 0.1756254451
[1 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
30        0.496992032203                0.490548714396                0.492097995619                1787953.26398
The NDCG socre on test set : 0.040553777665
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
31        0.519229705894                0.515869743559                0.513160346305                1582908.39998
The NDCG socre on test set : 0.355088083166
[3 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 3 1 1 1 3 1 1 1 1 1 1 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
32        0.407889280244                0.405591284499                0.409107282597                2155456.08453
The NDCG socre on test set : 0.44769734353
[2 0 3 0 0 0 0 3 0 0 3 3 3 0 3 3 0 0 2 0 0 0 2 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 

59        0.514130260488                0.510696721486                0.507674280911                1687000.59269
The NDCG socre on test set : 1.07410276289
[2 0 2 0 0 2 0 2 0 0 2 2 2 0 2 2 0 0 2 0 0 0 2 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
60        0.264115054768                0.267403159922                0.265186050074                2777916.88683
The NDCG socre on test set : 0.281824276233
[0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
61        0.442023355985                0.43685044993                 0.431109510146                2005582.49228
The NDCG socre on test set : 0.225436141823
[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 3 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
62        0.499930883093                0.494701584211                0.493543004542                1455337.02959
The NDCG socre on test set : 0.3378313682

The NDCG socre on test set : 0.559820717019
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
89        0.497311352314                0.495402938889                0.490102309944                1253188.94739
The NDCG socre on test set : 0.419194456366
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
90        0.37715022698                 0.372704126091                0.376749847839                2563794.87363
The NDCG socre on test set : 0.776623608146
[0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 0 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
91        0.255212797134                0.2529340004                  0.255841106984                2990025.56736
The NDCG socre on test set : 0.140012103254
[0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1